# <span style="color:#bce35b;font-size:35px"><b>Week 7 & 8</b></span>
<!-- # <span style="color:#bce35b;font-size:35px">Title<b>Title</b></span> -->
<!-- # <span style="color:#bce35b">H1</span> -->
<!-- ## <span style="color:#ae8bd5">H2</span> -->
<!-- ### <span style="color:#9c4957">H3</span> -->
<!-- #### <span style="color:white">H4</span> -->

<span style="color:#8c8c8c">Patrick Weatherford</span>

[Green]: <> (#bce35b)
[Purple]: <> (#ae8bd5)
[Coral]: <> (#9c4957)
[Grey]: <> (#8c8c8c)

***
***

Importing libraries

In [53]:
from datetime import date, datetime as dttm, timedelta
from pprint import pprint
from contextlib import closing
from dask import dataframe as dd
import datetime as dt
import sqlite3
from fuzzywuzzy import fuzz, process
import lxml
from bs4 import BeautifulSoup
import sys
import scipy
import io
import math
import re
from itertools import permutations, dropwhile, zip_longest
from collections import deque as deq
import requests
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np
import os
import keyring

os.environ['KAGGLE_USERNAME'] = 'hakunapatata'
os.environ['KAGGLE_KEY'] = keyring.get_password(
    service_name='kaggle_api', username=os.environ['KAGGLE_USERNAME'])


%matplotlib inline

# matplotlib default settings
plt.style.use('dark_background')
mpl.rcParams.update({'lines.linewidth': 3})
mpl.rcParams.update({'axes.labelsize': 14})
mpl.rcParams.update({'axes.titlesize': 16})
mpl.rcParams.update({'axes.titleweight': 'bold'})
mpl.rcParams.update({'figure.autolayout': True})
mpl.rcParams.update(
    {'axes.grid': True, 'grid.color': '#424242', 'grid.linestyle': '--'})

# creation random number generator object
rng_seed = 777
rng = np.random.default_rng(rng_seed)

# pandas dataframe options
pd.set_option('display.max_columns', None)


In [54]:
# see list of current settings
# plt.rcParams.keys()


<br>

***

# <span style="color:#bce35b">Week Activities</span>
<!-- # <span style="color:#bce35b;font-size:35px">Title<b>Title</b></span> -->
<!-- # <span style="color:#bce35b">H1</span> -->
<!-- ## <span style="color:#ae8bd5">H2</span> -->
<!-- ### <span style="color:#9c4957">H3</span> -->
<!-- #### <span style="color:white">H4</span> -->

<!-- <span style="color:#8c8c8c">Patrick Weatherford</span> -->

[Green]: <> (#bce35b)
[Purple]: <> (#ae8bd5)
[Coral]: <> (#9c4957)
[Grey]: <> (#8c8c8c)

From the datasets provided, manipulate/clean the dataset using 2 concepts from each of the chapters below.

<span style="font-size:12px">

Chapter 7
- <span style="color:#bce35b">Filter out missing data</span>
- Fill in missing data
- Remove duplicates
- Transform data using either mapping or a function
- Replace values
- Discretization and Binning
- <span style="color:#bce35b">Manipulate Strings</span>

Chapter 8
- Create hierarchical index
- <span style="color:#bce35b">Combine and Merge Datasets</span>
- Reshape
- <span style="color:#bce35b">Pivot the data</span>

Chapter 10
- Grouping with Dicts/Series
- <span style="color:#bce35b">Grouping with Functions</span>
- Grouping with Index Levels
- <span style="color:#bce35b">Split/Apply/Combine</span>
- Cross Tabs

Chapter 11
- <span style="color:#bce35b">Convert between string and date time</span>
- <span style="color:#bce35b">Generate date range</span>
- Frequencies and date offsets
- Convert timestamps to periods and back
- Period Frequency conversions

</span>

Import Metropolitan Museum of Art .csv file into a dask dataframe. Went with dask dataframe over pandas because csv was very large and dask incorporates lazy execution along with multi-core parallel processing. 

In [55]:
csv_url = "https://media.githubusercontent.com/media/metmuseum/openaccess/master/MetObjects.csv"

bad_line_list = []


def bad_line_func(line):
    bad_line_list.append(line)


df = dd.read_csv(csv_url, dtype="string",
                 on_bad_lines=bad_line_func, engine="python")


<br>

Import dask dataframe data into SQLite database/table. Went with the approace of importing into SQL database/table so that I could efficiently explore, extract, and manipulate the data.

In [56]:
db_path = 'HakunaPatata.db'
table_name = 'MET_MUSEUM'

df.to_sql(table_name, f"sqlite:///{db_path}", index=False,
          if_exists='replace', parallel=False, chunksize=5000)  # sqlite lacks parallel writing. would need to use a different database if parallel processing required


<br>

Create function for select statement into pandas DF.

In [57]:
def sql_df(db_path, sql_txt):

    with sqlite3.connect(db_path) as conn:
        df = pd.read_sql_query(sql_txt, conn)
        return df
    conn.close()


In [58]:

sql_txt = f"""
SELECT *
FROM MET_MUSEUM AS MM
LIMIT 5
"""

sql_df('HakunaPatata.db', sql_txt)


,Object Number,Is Highlight,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,Department,AccessionYear,Object Name,Title,Culture,Period,Dynasty,Reign,Portfolio,Constituent ID,Artist Role,Artist Prefix,Artist Display Name,Artist Display Bio,Artist Suffix,Artist Alpha Sort,Artist Nationality,Artist Begin Date,Artist End Date,Artist Gender,Artist ULAN URL,Artist Wikidata URL,Object Date,Object Begin Date,Object End Date,Medium,Dimensions,Credit Line,Geography Type,City,State,County,Country,Region,Subregion,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL
0,1979.486.1,False,False,False,1,None,The American Wing,1979,Coin,One-dollar Liberty Head Coin,None,None,None,None,None,16429,Maker,,James Barton Longacre,"American, Delaware County, Pennsylvania 1794–1...",,"Longacre, James Barton",American,1794,1869,None,http://vocab.getty.edu/page/ulan/500011409,https://www.wikidata.org/wiki/Q3806459,1853,1853,1853,Gold,Dimensions unavailable,"Gift of Heinz L. Stoppelmann, 1979",None,None,None,None,None,None,None,None,None,None,None,None,None,http://www.metmuseum.org/art/collection/search/1,None,None,"Metropolitan Museum of Art, New York, NY",None,None,None
1,1980.264.5,False,False,False,2,None,The American Wing,1980,Coin,Ten-dollar Liberty Head Coin,None,None,None,None,None,107,Maker,,Christian Gobrecht,1785–1844,,"Gobrecht, Christian",,1785,1844,None,http://vocab.getty.edu/page/ulan/500077295,https://www.wikidata.org/wiki/Q5109648,1901,1901,1901,Gold,Dimensions unavailable,"Gift of Heinz L. Stoppelmann, 1980",None,None,None,None,None,None,None,None,None,None,None,None,None,http://www.metmuseum.org/art/collection/search/2,None,None,"Metropolitan Museum of Art, New York, NY",None,None,None
2,67.265.9,False,False,False,3,None,The American Wing,1967,Coin,Two-and-a-Half Dollar Coin,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",None,None,None,None,None,None,None,None,None,None,None,None,None,http://www.metmuseum.org/art/collection/search/3,None,None,"Metropolitan Museum of Art, New York, NY",None,None,None
3,67.265.10,False,False,False,4,None,The American Wing,1967,Coin,Two-and-a-Half Dollar Coin,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",None,None,None,None,None,None,None,None,None,None,None,None,None,http://www.metmuseum.org/art/collection/search/4,None,None,"Metropolitan Museum of Art, New York, NY",None,None,None
4,67.265.11,False,False,False,5,None,The American Wing,1967,Coin,Two-and-a-Half Dollar Coin,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",None,None,None,None,None,None,None,None,None,None,None,None,None,http://www.metmuseum.org/art/collection/search/5,None,None,"Metropolitan Museum of Art, New York, NY",None,None,None


<br>

Now lets find some columns with NULL values and filter them out.

In [59]:
null_col_dict = {}

for col in df.columns:
    sql_txt = f"""
        WITH NULL_COUNTS AS (
        SELECT 
            SUM(CASE WHEN MM.\"{col}\" IS NULL OR MM.\"{col}\"='None' OR REPLACE(MM.\"{col}\", ' ', '')='' THEN 1 ELSE 0 END) AS NULL_COUNT
            , SUM(CASE WHEN MM.\"{col}\" IS NOT NULL AND MM.\"{col}\"<>'None' AND REPLACE(MM.\"{col}\", ' ', '')<>'' THEN 1 ELSE 0 END) AS NOT_NULL_COUNT 
        FROM MET_MUSEUM AS MM
        )
        
        SELECT 
            NC.NULL_COUNT
            , NC.NOT_NULL_COUNT
            , ROUND(100 * NC.NULL_COUNT / (NC.NOT_NULL_COUNT + NC.NULL_COUNT), 1) AS NULL_PERCENT
        FROM NULL_COUNTS AS NC
        """
    null_df = sql_df('HakunaPatata.db', sql_txt=sql_txt)

    null_col_dict[col] = (null_df['NULL_COUNT'][0],
                          null_df['NOT_NULL_COUNT'][0], null_df['NULL_PERCENT'][0])

print(f'Column\tNullCount\tNotNullCount\tNull%')
pprint(null_col_dict)


Column	NullCount	NotNullCount	Null%
{'AccessionYear': (3557, 474247, 0.0),
 'Artist Alpha Sort': (202441, 275363, 42.0),
 'Artist Begin Date': (233688, 244116, 48.0),
 'Artist Display Bio': (238475, 239329, 49.0),
 'Artist Display Name': (202269, 275535, 42.0),
 'Artist End Date': (233718, 244086, 48.0),
 'Artist Gender': (374744, 103060, 78.0),
 'Artist Nationality': (270059, 207745, 56.0),
 'Artist Prefix': (345839, 131965, 72.0),
 'Artist Role': (204369, 273435, 42.0),
 'Artist Suffix': (379148, 98656, 79.0),
 'Artist ULAN URL': (255784, 222020, 53.0),
 'Artist Wikidata URL': (260073, 217731, 54.0),
 'City': (445397, 32407, 93.0),
 'Classification': (78207, 399597, 16.0),
 'Constituent ID': (202269, 275535, 42.0),
 'Country': (402053, 75751, 84.0),
 'County': (469354, 8450, 98.0),
 'Credit Line': (452, 477352, 0.0),
 'Culture': (270425, 207379, 56.0),
 'Department': (1, 477803, 0.0),
 'Dimensions': (75295, 402509, 15.0),
 'Dynasty': (454571, 23233, 95.0),
 'Excavation': (461246, 165

<br>

Excluding Null values for ACCESSION_YEAR and manipulating strings so that if it is a blank string or equals 'None', then will output a Null value.

In [60]:
db_path = 'HakunaPatata.db'

sql_txt = r'''
SELECT 
    MM."Object Number" AS OBJECT_NUMBER
    , MM."AccessionYear" AS ACCESSION_YEAR
    , MM."Object Name" AS OBJECT_NAME
    , MM."Title" AS TITLE
    , MM."Object Date" AS OBJECT_DATE
    , MM."Object Begin Date" AS OBJECT_BEGIN_DATE
    , MM."Object End Date" AS OBJECT_END_DATE
    , MM."Artist Alpha Sort" AS ARTIST_NAME
    , MM."Artist Gender" AS ARTIST_GENDER
    , MM."Artist Nationality" AS ARTIST_NATIONALITY
    , MM."Credit Line" AS CREDIT_LINE
    , MM."Dimensions" AS DIMENSIONS
    , MM."Medium" AS MEDIUM
    , MM."Classification" AS CLASSIFICATION
    , MM."Department" AS DEPARTMENT
    , MM."Culture" AS CULTURE

FROM MET_MUSEUM AS MM
WHERE 
    MM."AccessionYear" <> 'None' AND REPLACE(MM."AccessionYear", ' ', '')<>'' -- should exclude Null values too
'''

met_df = sql_df(db_path, sql_txt)

# if 'None' string, None, or blank string then nan
met_df.replace({'None': np.nan, '': np.nan, None: np.nan}, inplace=True)
# exclude strings that are only white-space with nan
met_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

met_df


,OBJECT_NUMBER,ACCESSION_YEAR,OBJECT_NAME,TITLE,OBJECT_DATE,OBJECT_BEGIN_DATE,OBJECT_END_DATE,ARTIST_NAME,ARTIST_GENDER,ARTIST_NATIONALITY,CREDIT_LINE,DIMENSIONS,MEDIUM,CLASSIFICATION,DEPARTMENT,CULTURE
0,1979.486.1,1979,Coin,One-dollar Liberty Head Coin,1853,1853,1853,"Longacre, James Barton",NaN,American,"Gift of Heinz L. Stoppelmann, 1979",Dimensions unavailable,Gold,NaN,The American Wing,NaN
1,1980.264.5,1980,Coin,Ten-dollar Liberty Head Coin,1901,1901,1901,"Gobrecht, Christian",NaN,NaN,"Gift of Heinz L. Stoppelmann, 1980",Dimensions unavailable,Gold,NaN,The American Wing,NaN
2,67.265.9,1967,Coin,Two-and-a-Half Dollar Coin,1909–27,1909,1927,NaN,NaN,NaN,"Gift of C. Ruxton Love Jr., 1967",Diam. 11/16 in. (1.7 cm),Gold,NaN,The American Wing,NaN
3,67.265.10,1967,Coin,Two-and-a-Half Dollar Coin,1909–27,1909,1927,NaN,NaN,NaN,"Gift of C. Ruxton Love Jr., 1967",Diam. 11/16 in. (1.7 cm),Gold,NaN,The American Wing,NaN
4,67.265.11,1967,Coin,Two-and-a-Half Dollar Coin,1909–27,1909,1927,NaN,NaN,NaN,"Gift of C. Ruxton Love Jr., 1967",Diam. 11/16 in. (1.7 cm),Gold,NaN,The American Wing,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474242,23.112.2893,1923,Drawing,Phaeton #24567,1904,1904,1904,Brewster & Co.,NaN,NaN,"Gift of William Brewster, 1923",Sheet: 13 1/2 × 22 1/2 in. (34.3 × 57.2 cm),"Graphite on cardstock adhered to canvas, perfo...",Drawings,Drawings and Prints,NaN
474243,23.112.2894,1923,Drawing,Stanhope Phaeton #25538-25539 (#21222),1904,1904,1904,Brewster & Co.,NaN,NaN,"Gift of William Brewster, 1923",Sheet: 13 in. × 22 1/4 in. (33 × 56.5 cm),Graphite and ink,Drawings,Drawings and Prints,NaN
474244,53.600.1434,1953,Print,"Forest landscape with cattle drinking, a woman...",after 1773,1768,1778,"Sandby, Paul",NaN,NaN,"Harris Brisbane Dick Fund, 1953",Plate: 11 5/8 × 7 5/8 in. (29.6 × 19.4 cm),Etching,Prints,Drawings and Prints,NaN
474245,23.112.2895,1923,Drawing,Phaeton with folding top,1904,1890,1900,Brewster & Co.,NaN,NaN,"Gift of William Brewster, 1923",Sheet: 13 in. × 22 1/4 in. (33 × 56.5 cm),Graphite,Drawings,Drawings and Prints,NaN


<br>

Now let's pivot some data and join it onto the DataFrame.

Will group objects by begin date and end date and get totals for each. Will then join the grouped data onto the main DataFrame.

In [61]:
begin_date_totals_df = pd.DataFrame(met_df.groupby(
    ['OBJECT_BEGIN_DATE']).size(), columns=['OBJ_BEGIN_DATE_TOTAL'])


In [62]:
met_df = met_df.merge(begin_date_totals_df, how='left',
                      left_on='OBJECT_BEGIN_DATE', right_index=True)

met_df


,OBJECT_NUMBER,ACCESSION_YEAR,OBJECT_NAME,TITLE,OBJECT_DATE,OBJECT_BEGIN_DATE,OBJECT_END_DATE,ARTIST_NAME,ARTIST_GENDER,ARTIST_NATIONALITY,CREDIT_LINE,DIMENSIONS,MEDIUM,CLASSIFICATION,DEPARTMENT,CULTURE,OBJ_BEGIN_DATE_TOTAL
0,1979.486.1,1979,Coin,One-dollar Liberty Head Coin,1853,1853,1853,"Longacre, James Barton",NaN,American,"Gift of Heinz L. Stoppelmann, 1979",Dimensions unavailable,Gold,NaN,The American Wing,NaN,878
1,1980.264.5,1980,Coin,Ten-dollar Liberty Head Coin,1901,1901,1901,"Gobrecht, Christian",NaN,NaN,"Gift of Heinz L. Stoppelmann, 1980",Dimensions unavailable,Gold,NaN,The American Wing,NaN,650
2,67.265.9,1967,Coin,Two-and-a-Half Dollar Coin,1909–27,1909,1927,NaN,NaN,NaN,"Gift of C. Ruxton Love Jr., 1967",Diam. 11/16 in. (1.7 cm),Gold,NaN,The American Wing,NaN,2231
3,67.265.10,1967,Coin,Two-and-a-Half Dollar Coin,1909–27,1909,1927,NaN,NaN,NaN,"Gift of C. Ruxton Love Jr., 1967",Diam. 11/16 in. (1.7 cm),Gold,NaN,The American Wing,NaN,2231
4,67.265.11,1967,Coin,Two-and-a-Half Dollar Coin,1909–27,1909,1927,NaN,NaN,NaN,"Gift of C. Ruxton Love Jr., 1967",Diam. 11/16 in. (1.7 cm),Gold,NaN,The American Wing,NaN,2231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474242,23.112.2893,1923,Drawing,Phaeton #24567,1904,1904,1904,Brewster & Co.,NaN,NaN,"Gift of William Brewster, 1923",Sheet: 13 1/2 × 22 1/2 in. (34.3 × 57.2 cm),"Graphite on cardstock adhered to canvas, perfo...",Drawings,Drawings and Prints,NaN,664
474243,23.112.2894,1923,Drawing,Stanhope Phaeton #25538-25539 (#21222),1904,1904,1904,Brewster & Co.,NaN,NaN,"Gift of William Brewster, 1923",Sheet: 13 in. × 22 1/4 in. (33 × 56.5 cm),Graphite and ink,Drawings,Drawings and Prints,NaN,664
474244,53.600.1434,1953,Print,"Forest landscape with cattle drinking, a woman...",after 1773,1768,1778,"Sandby, Paul",NaN,NaN,"Harris Brisbane Dick Fund, 1953",Plate: 11 5/8 × 7 5/8 in. (29.6 × 19.4 cm),Etching,Prints,Drawings and Prints,NaN,369
474245,23.112.2895,1923,Drawing,Phaeton with folding top,1904,1890,1900,Brewster & Co.,NaN,NaN,"Gift of William Brewster, 1923",Sheet: 13 in. × 22 1/4 in. (33 × 56.5 cm),Graphite,Drawings,Drawings and Prints,NaN,5398


<br>

Doing split/apply/combine to get the total per object end date

In [63]:
def group_count_df(df, col):
    return df[col].count()


end_date_totals_df = pd.DataFrame(met_df.groupby(['OBJECT_END_DATE']).apply(
    group_count_df, col='OBJECT_END_DATE'), columns=['OBJ_END_DATE_TOTAL'])

met_df = met_df.merge(end_date_totals_df, how='left',
                      left_on='OBJECT_END_DATE', right_index=True)

met_df


,OBJECT_NUMBER,ACCESSION_YEAR,OBJECT_NAME,TITLE,OBJECT_DATE,OBJECT_BEGIN_DATE,OBJECT_END_DATE,ARTIST_NAME,ARTIST_GENDER,ARTIST_NATIONALITY,CREDIT_LINE,DIMENSIONS,MEDIUM,CLASSIFICATION,DEPARTMENT,CULTURE,OBJ_BEGIN_DATE_TOTAL,OBJ_END_DATE_TOTAL
0,1979.486.1,1979,Coin,One-dollar Liberty Head Coin,1853,1853,1853,"Longacre, James Barton",NaN,American,"Gift of Heinz L. Stoppelmann, 1979",Dimensions unavailable,Gold,NaN,The American Wing,NaN,878,857
1,1980.264.5,1980,Coin,Ten-dollar Liberty Head Coin,1901,1901,1901,"Gobrecht, Christian",NaN,NaN,"Gift of Heinz L. Stoppelmann, 1980",Dimensions unavailable,Gold,NaN,The American Wing,NaN,650,521
2,67.265.9,1967,Coin,Two-and-a-Half Dollar Coin,1909–27,1909,1927,NaN,NaN,NaN,"Gift of C. Ruxton Love Jr., 1967",Diam. 11/16 in. (1.7 cm),Gold,NaN,The American Wing,NaN,2231,883
3,67.265.10,1967,Coin,Two-and-a-Half Dollar Coin,1909–27,1909,1927,NaN,NaN,NaN,"Gift of C. Ruxton Love Jr., 1967",Diam. 11/16 in. (1.7 cm),Gold,NaN,The American Wing,NaN,2231,883
4,67.265.11,1967,Coin,Two-and-a-Half Dollar Coin,1909–27,1909,1927,NaN,NaN,NaN,"Gift of C. Ruxton Love Jr., 1967",Diam. 11/16 in. (1.7 cm),Gold,NaN,The American Wing,NaN,2231,883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474242,23.112.2893,1923,Drawing,Phaeton #24567,1904,1904,1904,Brewster & Co.,NaN,NaN,"Gift of William Brewster, 1923",Sheet: 13 1/2 × 22 1/2 in. (34.3 × 57.2 cm),"Graphite on cardstock adhered to canvas, perfo...",Drawings,Drawings and Prints,NaN,664,767
474243,23.112.2894,1923,Drawing,Stanhope Phaeton #25538-25539 (#21222),1904,1904,1904,Brewster & Co.,NaN,NaN,"Gift of William Brewster, 1923",Sheet: 13 in. × 22 1/4 in. (33 × 56.5 cm),Graphite and ink,Drawings,Drawings and Prints,NaN,664,767
474244,53.600.1434,1953,Print,"Forest landscape with cattle drinking, a woman...",after 1773,1768,1778,"Sandby, Paul",NaN,NaN,"Harris Brisbane Dick Fund, 1953",Plate: 11 5/8 × 7 5/8 in. (29.6 × 19.4 cm),Etching,Prints,Drawings and Prints,NaN,369,342
474245,23.112.2895,1923,Drawing,Phaeton with folding top,1904,1890,1900,Brewster & Co.,NaN,NaN,"Gift of William Brewster, 1923",Sheet: 13 in. × 22 1/4 in. (33 × 56.5 cm),Graphite,Drawings,Drawings and Prints,NaN,5398,13133


<br>

Create a date column from the OBJECT_BEGIN_DATE column.

- Will need to handle B.C. dates. Will convert negative (B.C.) years to positive and will create a column to categorize if the year is B.C. vs. A.D.
- Also, the datetime module does not go past range 9999 (aka year 9999). Even when handling via converting to positive year, an out-of-range exception will be thrown. I started going down the path of looking into the package astropy.time which is used for astrophysics but after about 2 hrs of persistent desk head banging I decided that I'm probably overthinking the assignment I or may be trying to find a solution to something that does not exist. So, if the absolute value of the year is > 9999, then NaT will be output.

In [64]:
# function to handle B.C. dates. Will convert negative years to postive and then convert to date.
def year_str_to_dttm(string):
    try:
        year_num = int(string)
        if year_num < 0:
            year_str = str(int(-1 * year_num))
            date_year = dttm.strptime(year_str, '%Y')
            return date_year
        else:
            return dttm.strptime(string, '%Y')

    except Exception:
        return pd.NaT


# function to create column that specifies if the year is B.C. or A.D.
def year_bc_ad(string):
    try:
        year_num = int(string)
        if year_num < 0:
            return 'B.C.'

        else:
            return 'A.D.'

    except Exception:
        return np.nan


In [65]:
# using apply, use formula created above to create a date column based on the object begin year
met_df['OBJ_BEGIN_YEAR_TRUNC'] = met_df['OBJECT_BEGIN_DATE'].apply(
    year_str_to_dttm)

# using apply, use formula created above to create a date column based on the object begin year
met_df['OBJ_BEGIN_YEAR_BC_AD'] = met_df['OBJECT_BEGIN_DATE'].apply(year_bc_ad)

met_df[met_df['OBJ_BEGIN_YEAR_BC_AD'] == 'B.C.']


,OBJECT_NUMBER,ACCESSION_YEAR,OBJECT_NAME,TITLE,OBJECT_DATE,OBJECT_BEGIN_DATE,OBJECT_END_DATE,ARTIST_NAME,ARTIST_GENDER,ARTIST_NATIONALITY,CREDIT_LINE,DIMENSIONS,MEDIUM,CLASSIFICATION,DEPARTMENT,CULTURE,OBJ_BEGIN_DATE_TOTAL,OBJ_END_DATE_TOTAL,OBJ_BEGIN_YEAR_TRUNC,OBJ_BEGIN_YEAR_BC_AD
2894,27.17.39,1927,Plate,Plate,ca..1770,-5,5,Bow Porcelain Factory,NaN,British,"Rogers Fund, 1927",Diam. 8 in. (20.3 cm),Soft-paste,NaN,The American Wing,British,12,22,NaT,B.C.
17193,06.310.23,1906,Ceremonial spearhead (<i>Tsukushi boko</i>),Ceremonial Spearhead (&lt;i&gt;Tsukushi Boko&l...,probably 1st–2nd century A.D.,-25,225,NaN,NaN,NaN,"Gift of Bashford Dean, by exchange, 1906",L. 33 1/2 in. (85.1 cm); W. 5 1/4 in. (13.3 cm...,Bronze,Shafted Weapons,Arms and Armor,Japanese,466,62,NaT,B.C.
17239,12.37.110–.119,1912,Ten spearheads and sinkers,Ten Spearheads and Sinkers,ca. 12500 B.C.–ca. 4th or 3rd century B.C.,-12500,-300,NaN,NaN,NaN,"Rogers Fund, 1912\r\n",12.37.110; L. 4 3/4 in. (12.1 cm); W. 3 1/2 in...,Stone,Miscellaneous,Arms and Armor,Japanese,2,4628,NaT,B.C.
17240,12.37.100–.109,1912,Ten spearheads and sinkers,Ten Spearheads and Sinkers,ca. 12500 B.C.–ca. 4th or 3rd century B.C.,-12500,-300,NaN,NaN,NaN,"Rogers Fund, 1912",12.37.100; L. 4 1/2 in. (11.4 cm); W. 2 1/4 in...,Stone,Miscellaneous,Arms and Armor,Japanese,2,4628,NaT,B.C.
17241,12.37.91–.99,1912,Nine celts or polished fleshers,Nine Celts or Polished Fleshers,probably ca. 10th century B.C.,-1050,-850,NaN,NaN,NaN,"Rogers Fund, 1912\r\n",12.27.91; L. 7 in. (17.8 cm); W. 3/4 in. (1.9 ...,Stone,Archery Equipment-Bows,Arms and Armor,Japanese,457,16,1050-01-01 00:00:00,B.C.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472936,20.4c,1920,"Coffin, Lady Nefer, mummy",Mummy of the Lady Nefer,4rd century B.C. or later,-400,-200,NaN,NaN,NaN,"Gift of John W. Baxter, 1920",NaN,"Wood, gesso, paint, human remains",NaN,Egyptian Art,NaN,435,963,NaT,B.C.
473022,2021.259,2021,Wine container (hu),NaN,5th–4th century B.C.,-499,-300,NaN,NaN,NaN,"Purchase, The Vincent Astor Foundation Gift, 2021",H. 13 in. (33 cm),Bronze inlaid with bronze,Metalwork,Asian Art,China,589,4628,NaT,B.C.
474013,74.51.6718,1874,Askos,Terracotta askos in the shape of a quadruped,ca. 2000–1725 B.C.,-2000,-1725,NaN,NaN,NaN,"The Cesnola Collection, Purchased by subscript...","2 5/8 × 1 1/2 × 3 3/4 in., 1.764oz. (6.7 × 3.8...",Terracotta,Vases,Greek and Roman Art,Cypriot,519,35,2000-01-01 00:00:00,B.C.
474015,74.51.6719,1874,Askos,Terracotta askos in the shape of a water bird,early 11th century B.C.,-1100,-1001,NaN,NaN,NaN,"The Cesnola Collection, Purchased by subscript...","4 1/2 × 2 3/4 × 6 1/4 in., 5.114oz. (11.4 × 7 ...",Terracotta,Vases,Greek and Roman Art,Cypriot,42,1,1100-01-01 00:00:00,B.C.


<br>

Create date range from min/max date values from OBJ_BEGIN_YEAR_TRUNC. As mentioned above, the datetime module has a range of [1, 9999] so can only make a date range from year 1 A.D. to 9999 A.D. Now I'm starting to see why there was the Y2K problem in year 2000 lol.

In [66]:
begin_date = dttm.strptime('0001', '%Y')
end_date = max(met_df[met_df['OBJ_BEGIN_YEAR_BC_AD'] == 'A.D.']['OBJ_BEGIN_YEAR_TRUNC'])


date_list = []
for i in range(begin_date.year, end_date.year+1):
    year_str = f"{i:04d}"
    date_time = dttm.strptime(year_str, '%Y')
    date_list.append(date_time)


pd.DataFrame(date_list, columns=['OBJ_BEGIN_YEAR'], index=[pd.Index(date_list, name='OBJ_BEGIN_YEAR')])

,OBJ_BEGIN_YEAR
OBJ_BEGIN_YEAR,
0001-01-01 00:00:00,0001-01-01 00:00:00
0002-01-01 00:00:00,0002-01-01 00:00:00
0003-01-01 00:00:00,0003-01-01 00:00:00
0004-01-01 00:00:00,0004-01-01 00:00:00
0005-01-01 00:00:00,0005-01-01 00:00:00
...,...
4996-01-01 00:00:00,4996-01-01 00:00:00
4997-01-01 00:00:00,4997-01-01 00:00:00
4998-01-01 00:00:00,4998-01-01 00:00:00
